In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
url_jan = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"
url_feb = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet"

df_jan = pd.read_parquet(url_jan)
df_feb = pd.read_parquet(url_feb)


In [21]:
print("Number of columns in January 2023 data:", len(df_jan.columns))
print("Column names:")
print(df_jan.columns.tolist())

df_jan.info()
df_jan.head()

Number of columns in January 2023 data: 19
Column names:
['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'airport_fee']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           in

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [23]:
# # Combine both months
# df = pd.concat([df_jan, df_feb], ignore_index=True)

# # Convert datetime columns
# df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
# df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# # Compute trip duration in minutes
# df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# # Filter unreasonable durations
# df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# # Quick check
# df['duration'].describe()

# # Compute standard deviation
# std_duration = df['duration'].std()
# print(f"Standard deviation of trip duration: {std_duration:.2f}")

# Convert datetime columns
df_jan['tpep_pickup_datetime'] = pd.to_datetime(df_jan['tpep_pickup_datetime'])
df_jan['tpep_dropoff_datetime'] = pd.to_datetime(df_jan['tpep_dropoff_datetime'])

# Compute duration in minutes (no filtering)
df_jan['duration'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

# Compute standard deviation
std_duration = df_jan['duration'].std()
print(f"Standard deviation of trip duration in January: {std_duration:.2f}")

Standard deviation of trip duration in January: 42.59


In [13]:
# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']
# target = 'duration'

# df_model = df[categorical + numerical + [target]]

In [15]:
# df_model.to_parquet('processed_yellow_tripdata_2023_01_02.parquet', index=False)


In [25]:
# Filter out outliers: keep only durations between 1 and 60 minutes (inclusive)
initial_count = len(df_jan)
df_jan = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]
final_count = len(df_jan)

fraction = final_count / initial_count
print(f"Fraction of records kept: {fraction:.2f}")

Fraction of records kept: 0.98


In [39]:
# Q4
from sklearn.feature_extraction import DictVectorizer

# Convert location IDs to string
df_jan = df_jan.copy()  # to silence the warning
df_jan['PULocationID'] = df_jan['PULocationID'].astype(str)
df_jan['DOLocationID'] = df_jan['DOLocationID'].astype(str)

# Convert to list of dictionaries
train_dicts = df_jan[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Fit the DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Get matrix shape
print("Matrix shape:", X_train.shape)

Matrix shape: (3009173, 515)


In [49]:
# Q5
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Target variable
y_train = df_jan['duration'].values

# Train the model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict on training data
y_pred = lr.predict(X_train)

# Compute RMSE
rmse_train = mean_squared_error(y_train, y_pred, squared=False)
print(f"Q5 - RMSE on training data: {rmse_train:.2f}")

Q5 - RMSE on training data: 7.65


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [51]:
# Q6
# Load and preprocess February data
df_feb = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

# Convert datetime columns
df_feb['tpep_pickup_datetime'] = pd.to_datetime(df_feb['tpep_pickup_datetime'])
df_feb['tpep_dropoff_datetime'] = pd.to_datetime(df_feb['tpep_dropoff_datetime'])

# Compute duration
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

# Filter durations between 1 and 60 minutes
df_feb_filtered = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)].copy()

# Convert IDs to strings
df_feb_filtered['PULocationID'] = df_feb_filtered['PULocationID'].astype(str)
df_feb_filtered['DOLocationID'] = df_feb_filtered['DOLocationID'].astype(str)

# Turn into list of dictionaries
val_dicts = df_feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform using the DictVectorizer from training
X_val = dv.transform(val_dicts)
y_val = df_feb_filtered['duration'].values

# Predict and calculate RMSE
y_pred_val = lr.predict(X_val)
from sklearn.metrics import mean_squared_error
rmse_val = mean_squared_error(y_val, y_pred_val, squared=False)
print(f"Q6 - RMSE on validation data: {rmse_val:.2f}")

Q6 - RMSE on validation data: 7.81


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
